# Introduction

Notebook for scraping and create a bot to send them connects with custom messages in the sale navigator
linkedIn.

**Note:**
* This code will only work for the linkedin's sale navigator link (page source) after applying the filters
* This code will create a excel sheet and that simple code ftn can be modified to store other details too.
* This code is only for educational purpose.

## Installation

* Please first install the required libraries using: 
    ``pip install -r requirements.txt``
* Download chrome-driver of matching version with the chrome browser | https://chromedriver.chromium.org/downloads
* Place the driver in a folder and give the path as following.
* Add the username/email and password in the config.txt or you can add them in the cell too.

In [30]:
# import libraries and modules
import os, sys, time
from random import *
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
# add the driver path
driver = webdriver.Chrome(r'\driver\chromedriver.exe')

C:\Users\Ozzi\AppData\Local\Temp\ipykernel_2436\2141422295.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\Ozzi\Documents\LinkedIn_automation\driver\chromedriver.exe')


## Procedure

In [3]:
# open the linkedin login page
driver.get('https://www.linkedin.com/uas/login')

In [4]:
# read the username and password from the config file
line = open('config.txt', 'r').readlines()
username = line[0]
password = line[1]

In [6]:
# find the username and password fields and enter the credentials
elementID = driver.find_element(By.ID, 'username').send_keys(username)
elementID = driver.find_element(By.ID, 'password').send_keys(password)

**Add the sale navigator filtered link.**

The code will go the specific section and will scrap the details of the peoples, will store it
in a list and will also generate an excel sheet too.

In [7]:
# Add the link here:
driver.get('https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(id%3A2105439369%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3AREGION%2Cvalues%3AList((id%3A101282230%2Ctext%3AGermany%2CselectionType%3AINCLUDED)))%2C(type%3ASENIORITY_LEVEL%2Cvalues%3AList((id%3A8%2Ctext%3ACXO%2CselectionType%3AINCLUDED)%2C(id%3A6%2Ctext%3ADirector%2CselectionType%3AINCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A13%2Ctext%3AInformation%2520Technology%2CselectionType%3AINCLUDED)))%2C(type%3AYEARS_OF_EXPERIENCE%2Cvalues%3AList((id%3A5%2Ctext%3AMore%2520than%252010%2520years%2CselectionType%3AINCLUDED)))%2C(type%3APAST_COMPANY%2Cvalues%3AList((id%3A2508619%2Ctext%3ABosch%2CselectionType%3AINCLUDED)))))&sessionId=afxxwqmSQ%2FWpEUA3eSfzTg%3D%3D')

In [24]:
visitedProfile = []
profileQueued = []

In [22]:
def getNewProfileIDs(soup, profilesQueued):
    """
        ftn to create a list of new profiles to be scraped from the search results page,
        add them to the profilesQueued list and generate a excel file with the details too.

    Args:
        soup (object): page source of the search results page
        profilesQueued (list): list of profiles already queued for scraping

    Returns:
        list: list of new profiles to be scraped
    """
    
    profilesID = []
    names = []
    titles = []
    locations = []
    companies = []
    
    # emails = []
    # websites = []
    
    pav = soup.find('div', {'id': 'search-results-container'})
    all_links = pav.find_all('a', {'data-control-name': 'view_lead_panel_via_search_lead_name'})
    
    FullName = pav.find_all('span', {'data-anonymize': 'person-name'})
    Title = pav.find_all('span', {'data-anonymize': 'title'})
    Location = pav.find_all('span', {'data-anonymize': 'location'})
    company = pav.find_all('a', {'data-anonymize': 'company-name'})
    
    
    for i, link in enumerate(all_links):
        userID = link.get('href')
        if((userID not in profilesQueued) and (userID not in visitedProfile)):
            profilesID.append(userID)
            names.append(FullName[i].text)
            titles.append(Title[i].text)
            locations.append(Location[i].text)
            companies.append(company[i].text)
    
    df = pd.DataFrame({'FullName':names, 'Title':titles, 'Location':locations, 'Company':companies, 'ProfileID':profilesID})
    df.to_excel('profiles.xlsx', index=False)
    return profilesID

In [28]:
# get the new profiles from the search results page
profileQueued = getNewProfileIDs(BeautifulSoup(driver.page_source), profileQueued)

In [29]:
profileQueued, len(profileQueued)

(['/sales/lead/ACwAABqnREoBnyTo22gYT7gfkZm3k1k--kZQnwM,NAME_SEARCH,tb-M?_ntb=afxxwqmSQ%2FWpEUA3eSfzTg%3D%3D',
  '/sales/lead/ACwAADOgrPoBCMrbUMEelSgc5aEUuSIx_LGEthU,NAME_SEARCH,_a5E?_ntb=afxxwqmSQ%2FWpEUA3eSfzTg%3D%3D',
  '/sales/lead/ACwAAALwlG8BMEWE1ajbfh5Gjn1m6KlQyDpYAlo,NAME_SEARCH,WX9j?_ntb=afxxwqmSQ%2FWpEUA3eSfzTg%3D%3D'],
 3)

In [33]:
# go to the each profile 
# and send the connection request with a custom message and then pause for a random time
# and then go to the next profile

while profileQueued:
    try:
        visitingProfileID = profileQueued.pop()
        visitedProfile.append(visitingProfileID)

        fullLink = 'https://www.linkedin.com' + visitingProfileID
        driver.get(fullLink)
        driver.implicitly_wait(5)
        
        fullName = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[3]/div/div[1]/div/div/section[1]/section[1]/div[1]/div[2]/h1').text

        driver.find_element(By.XPATH,'/html/body/main/div[1]/div[3]/div/div[1]/div/div/section[1]/section[1]/div[2]/section/button').click()    

        driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/ul/li[1]/button').click()

        customMessage = f"Hello {fullName},\nI have found mutual interest area and I would be more than happy to connect with you.\nKindly, accept my invitation. Thanks!"

        elementID = driver.find_element(By.ID, 'connect-cta-form__invitation')
        elementID.send_keys(customMessage)


        driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div[3]/div/button[2]').click()
        

        # Add the ID to the visitedUsersFile
        with open('visitedUsers.txt', 'a') as visitedUsersFile:
            visitedUsersFile.write(str(visitingProfileID)+'\n')
        visitedUsersFile.close()

        # Pause
        #Defining a random value for our delay (15-20 minutes)
        sleepInt = randint(900, 1200)
        time.sleep(sleepInt)

        if(len(visitedProfiles)%50==0):
            print('Visited Profiles: ', len(visitedProfiles))

    except:
        print('error')